In [1]:
import math
import os.path

import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import activations
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
import time
from skimage import transform, metrics
from umap import UMAP
import datetime
from scipy.signal import argrelextrema
import os
import pandas as pd
import scipy
from skimage import io
import csv
from sklearn.manifold import TSNE
from skimage.metrics import structural_similarity as ssim
import math
import plotly.express as px

from src.utils import GrainLogs
from src.nn import RSU7, RSU6, RSU5, RSU4, RSU4F, ConvBlock
from src.nn_utils import SaveImageCallback

gpus = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpus[0], True)

In [3]:
class Encoder(tf.keras.Model):

    def __init__(self, latent_dim, label_dim):
        super(Encoder, self).__init__()

        self.latent_dim = latent_dim
        self.label_dim = label_dim

        self.stage1 = RSU7(16, 32)
        self.pool12 = layers.MaxPool2D((2, 2), 2)

        self.stage2 = RSU6(32, 64)
        self.pool23 = layers.MaxPool2D((2, 2), 2)

        self.stage3 = RSU5(64, 128)
        self.pool34 = layers.MaxPool2D((2, 2), 2)

        self.stage4 = RSU4(128, 256)
        self.pool45 = layers.MaxPool2D((2, 2), 2)
        #out_w_h=64

        self.stage5 = RSU4F(256, 256)
        self.pool56 = layers.MaxPool2D((2, 2), 2)

    def __call__(self, inputs):
        #Encoder block 1

        hx1 = self.stage1(inputs)

        hx = self.pool12(hx1)
        hx2 = self.stage2(hx)
        hx = self.pool23(hx2)

        hx3 = self.stage3(hx)
        hx = self.pool34(hx3)

        hx4 = self.stage4(hx)
        hx = self.pool45(hx4)

        hx5 = self.stage5(hx)
        x = self.pool56(hx5)
        global_pool = layers.GlobalAveragePooling2D()(x)

        x = tf.keras.layers.Flatten()(global_pool)
        x = tf.keras.layers.Dense(512)(x)
        x = layers.LeakyReLU(0.2)(x)

        x = tf.keras.layers.Dense(self.latent_dim + self.latent_dim)(x)
        x = layers.LeakyReLU(0.2)(x)

        x_label = layers.Dense(128)(global_pool)
        x_label = layers.LeakyReLU()(x_label)
        x_label = layers.Dense(self.label_dim, name='encoder_label_output')(x_label)
        x_label = layers.LeakyReLU()(x_label)

        # tf.keras.layers.InputLayer(input_shape=(28, 28, 1))
        x = tf.keras.layers.Conv2D(
            filters=32, kernel_size=3, strides=(2, 2), activation='relu')(inputs)
        x = tf.keras.layers.Conv2D(
            filters=64, kernel_size=3, strides=(2, 2), activation='relu')(x)
        x = tf.keras.layers.Flatten()(x)
        # No activation
        x = tf.keras.layers.Dense(latent_dim + latent_dim)(x)

        # return x, x_label
        return x


In [4]:
class Decoder(tf.keras.Model):

    def __init__(self, label_dim, batch_size=32, out_ch=1):
        super(Decoder, self).__init__()

        self.batch_size = batch_size
        self.label_dim = label_dim
        self.out_ch = out_ch

        self.stage6 = RSU4F(256, 256)
        self.stage5d = RSU4F(128, 128)
        self.stage4d = RSU4(64, 64)
        self.stage3d = RSU5(32, 32)
        self.stage2d = RSU6(16, out_ch)
        self.stage1d = RSU7(16, out_ch)

    def __call__(self, z_inputs):
        # Reshape input
        # z_image_v, labels = tf.split(z_inputs, axis=1, num_or_size_splits=2)
        # z_data = tf.concat(z_inputs, axis=1)
        x = layers.Dense(1024)(z_inputs)
        x = tf.nn.leaky_relu(x, alpha=0.3)

        x = layers.Dense(512)(x)
        x = tf.nn.leaky_relu(x, alpha=0.3)

        x = layers.Dense(256)(x)
        x = tf.nn.leaky_relu(x, alpha=0.3)

        x = layers.Reshape(target_shape=(16, 16, 1))(x)

        x = layers.UpSampling2D(size=(2, 2), interpolation='bilinear')(x)
        x = RSU4F(256, 256)(x)

        x = layers.UpSampling2D(size=(2, 2), interpolation='bilinear')(x)
        x = RSU4F(128, 128)(x)

        x = layers.UpSampling2D(size=(2, 2), interpolation='bilinear')(x)
        x = RSU4(64, 64)(x)

        x = layers.UpSampling2D(size=(2, 2), interpolation='bilinear')(x)
        x = RSU5(32, 32)(x)

        x = layers.UpSampling2D(size=(2, 2), interpolation='bilinear')(x)
        x = RSU6(16, 16)(x)

        x = layers.UpSampling2D(size=(2, 2), interpolation='bilinear')(x)
        x = RSU6(8, self.out_ch)(x)

        x = tf.keras.layers.Dense(units=7 * 7 * 32, activation=tf.nn.relu)(z_inputs)
        x = tf.keras.layers.Reshape(target_shape=(7, 7, 32))(x)
        x = tf.keras.layers.Conv2DTranspose(
            filters=64, kernel_size=3, strides=2, padding='same',
            activation='relu')(x)
        x = tf.keras.layers.Conv2DTranspose(
            filters=32, kernel_size=3, strides=2, padding='same',
            activation='relu')(x)
        # No activation
        x = tf.keras.layers.Conv2DTranspose(
            filters=1, kernel_size=3, strides=1, padding='same')(x)

        # x = activations.sigmoid(x)

        # return x, x_label
        return x

In [ ]:
# latent_dim = (200,)
# label_dim = (5,)
#
# decoder = Decoder(label_dim=5)
# z_input = layers.Input(shape=latent_dim)
# label_input = layers.Input(shape=label_dim)
# outputs = decoder([z_input, label_input])
#
# model = tf.keras.Model(inputs=[z_input, label_input], outputs=outputs)
# model.summary()

In [ ]:
# model=tf.keras.applications.VGG19(include_top=False,input_shape=(64,64,3))
# tf.keras.utils.plot_model(model, to_file='cvae_1.png')

In [5]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
images_dataset = (x_train / 255).reshape((-1, 28, 28, 1))

In [ ]:
images = np.load('data/saved np/images_no_filters.npy')

grain_names = np.array(
    [['Ultra_Co8.jpg'], ['Ultra_Co11.jpg'], ['Ultra_Co6_2.jpg'], ['Ultra_Co15.jpg'], ['Ultra_Co25.jpg']])

labels = np.array([[91, 12.1, 1210],
                   [78, 8.1, 1180],
                   [62, 8.9, 1100],
                   [72, 21.6, 990],
                   [99, 15.3, 1200]])

In [ ]:
scaler = preprocessing.MinMaxScaler()
scaler.fit(labels)
labels = scaler.transform(labels)

labels_dataset = []
for i in range(images.shape[0]):
    for j in range(images.shape[1]):
        labels_dataset.append(labels[i])

images_dataset = images.reshape((-1, 1024, 1024, 1))

In [6]:
train_images, test_images = train_test_split(images_dataset, test_size=0.2, random_state=421)
# train_labels, test_labels = train_test_split(labels_dataset, test_size=0.2, random_state=421)


In [ ]:
# # Checkpoint path
# checkpoint_root = "./CVAE{}_{}_checkpoint".format(latent_dim, beta)
# checkpoint_name = "model"
# save_prefix = os.path.join(checkpoint_root, checkpoint_name)
#
# # Define the checkpoint
# checkpoint = tf.train.Checkpoint(module=cvae)

In [41]:

class ConvCVAE(tf.keras.Model):

    def __init__(self,
                 label_dim,
                 latent_dim,
                 beta=1,
                 batch_size=2
                 ):
        super(ConvCVAE, self).__init__()

        self.encoder = Encoder(latent_dim=latent_dim, label_dim=label_dim)
        self.decoder = Decoder(label_dim=label_dim)
        self.label_dim = label_dim
        self.latent_dim = latent_dim
        self.beta = beta
        self.batch_size = batch_size

    # @tf.function
    def sample(self, eps=None):
        if eps is None:
            eps = tf.random.normal(shape=(self.batch_size, self.latent_dim))
            # eps = tf.random.normal(shape=(self.latent_dim,))
        return self.decode(eps, apply_sigmoid=False)

    def encode(self, x):
        mean, logvar = tf.split(self.encoder(x), num_or_size_splits=2, axis=1)
        return mean, logvar

    def reparameterize(self, mean, logvar):

        eps = tf.random.normal(shape=(self.batch_size,self.latent_dim))
        # eps = tf.random.normal(shape=(self.latent_dim,))
        return eps * tf.exp(logvar * 0.5) + mean

    def decode(self, z, apply_sigmoid=False):
        logits = self.decoder(z)
        if apply_sigmoid:
            probs = tf.sigmoid(logits)
            return probs
        return logits

In [62]:
# def compute_loss(model, x):
#     z_mean, z_log_var = model.encode(x)
#     z = model.reparameterize(z_mean, z_log_var)
#     logits = model.decode(z, apply_sigmoid=False)
#     rec_loss = tf.cast(tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits, x)), tf.float32)
#     # rec_loss = tf.cast(tf.reduce_sum(tf.nn.sigmoid_cross_entropy_with_logits(logits=logits, labels=x), 1), tf.float32)
#     latent_loss = tf.cast(
#         tf.reduce_mean(-0.5 * tf.reduce_sum(1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var), axis=-1)),
#         tf.float32)
#
#     total_loss = rec_loss + beta * latent_loss
#     # print(
#     #     f'total loss {total_loss}, reconstr_loss {rec_loss}, latent_loss {latent_loss}')
#
#     return total_loss, rec_loss, latent_loss


def log_normal_pdf(sample, mean, logvar, raxis=1):
    log2pi = tf.math.log(2. * np.pi)
    return tf.reduce_sum(
        -.5 * ((sample - mean) ** 2. * tf.exp(-logvar) + logvar + log2pi),
        axis=raxis)

def compute_loss(model, x):
    mean, logvar = model.encode(x)
    z = model.reparameterize(mean, logvar)
    x_logit = model.decode(z)
    cross_ent = tf.nn.sigmoid_cross_entropy_with_logits(logits=x_logit, labels=x)
    logpx_z = -tf.reduce_sum(cross_ent, axis=[1, 2, 3])
    logpz = log_normal_pdf(z, 0., 0.)
    logqz_x = log_normal_pdf(z, mean, logvar)
    return -tf.reduce_mean(logpx_z + logpz - logqz_x),0,0


# @tf.function
def train_step(model, x, optimizer):
    """Executes one training step and returns the loss.

    This function computes the loss and gradients, and uses the latter to
    update the model's parameters.
    """

    with tf.GradientTape() as tape:
        total_loss, rec_loss, latent_loss = compute_loss(model, x)

    gradients = tape.gradient(total_loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    return total_loss, rec_loss, latent_loss


In [48]:
def generate_and_save_images(model, epoch, test_sample):
    mean, logvar = model.encode(test_sample)
    z = model.reparameterize(mean, logvar)
    logits = activations.sigmoid(model.sample(z))
    prediction = logits.numpy()[0] * 255
    io.imsave('image_at_epoch_{:04d}.png'.format(epoch), prediction.astype(np.uint8))

In [10]:
def printProgressBar(epoch, iteration, total, prefix='', suffix='', decimals=1, length=100, fill='█', printEnd="\r",
                     eta=None, loss=None, train_type='train'):
    percent = ("{0:." + str(decimals) + "f}").format(100 * (iteration / float(total)))
    filledLength = int(length * iteration // total)
    bar = fill * filledLength + '-' * (length - filledLength)
    print(
        f'\r{prefix} |{bar}| {percent}% {suffix} ETA:{eta} s epoch={epoch}: ELBO={str(np.round(loss[0], 4))},rec_loss={str(np.round(loss[1], 4))} lat_loss={str(round(loss[2], 4))}',
        end=printEnd)
    # Print New Line on Complete
    if iteration == total:
        print()


In [63]:
epochs = 100
# set the dimensionality of the latent space to a plane for visualization later
latent_dim = 200
label_dim = 3

model = ConvCVAE(label_dim, latent_dim,batch_size=32)

In [65]:
# generate_and_save_images(model, 0, test_sample)
optimizer = tf.keras.optimizers.RMSprop(learning_rate=1e-3,centered=True)
# optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
batch_size = 64
beta = 1

n_batches = train_images.shape[0] // batch_size
n_batches_test = test_images.shape[0] // batch_size
model.batch_size = batch_size

for epoch in range(1, epochs + 1):
    train_images_dataset = tf.data.Dataset.from_tensor_slices(train_images.astype(np.float32)).batch(
        batch_size).as_numpy_iterator()
    test_images_dataset = tf.data.Dataset.from_tensor_slices(test_images.astype(np.float32)).batch(
        batch_size).as_numpy_iterator()

    start_time = time.time()
    total_loss = []
    total_rec_loss = []
    total_latent_loss = []

    printProgressBar(epoch, 0, n_batches, eta=None, loss=[0, 0, 0], prefix='Progress:', suffix='Complete',
                     train_type='train', length=1)
    for i, train_x in enumerate(range(n_batches)):
        # print(f'epoch={epoch} batch num={i}/{n_batches}')

        loss, rec_loss, latent_loss = train_step(model, train_images_dataset.next(), optimizer)
        total_loss.append(loss)
        total_rec_loss.append(rec_loss)
        total_latent_loss.append(latent_loss)
        printProgressBar(epoch, i + 1, n_batches, eta=None, prefix='Progress:', suffix='Complete', train_type='train',
                         loss=[np.mean(total_loss), np.mean(rec_loss), np.mean(total_latent_loss)], length=1)

    end_time = time.time()

    loss = tf.keras.metrics.Mean()
    for i, test_x in enumerate(range(n_batches_test)):
        loss(compute_loss(model, test_images_dataset.next())[0])
    elbo = loss.result()
    # display.clear_output(wait=False)
    print('Epoch: {}, Test set ELBO: {}, time elapse for current epoch: {}'
          .format(epoch, elbo, end_time - start_time))
    test_sample = test_images[np.random.randint(0, test_images.shape[0])]
    generate_and_save_images(model, epoch, np.expand_dims(test_sample, axis=0).astype(np.float32))

Progress: |█| 100.0% Complete ETA:None s epoch=1: ELBO=543.9858,rec_loss=0.0 lat_loss=0.0
Epoch: 1, Test set ELBO: 542.7450561523438, time elapse for current epoch: 24.30491590499878
Progress: |█| 100.0% Complete ETA:None s epoch=2: ELBO=544.0162,rec_loss=0.0 lat_loss=0.0
Epoch: 2, Test set ELBO: 544.2404174804688, time elapse for current epoch: 24.031843185424805


<ipython-input-48-7578ee8870af>:6: UserWarning: image_at_epoch_0002.png is a low contrast image
  io.imsave('image_at_epoch_{:04d}.png'.format(epoch), prediction.astype(np.uint8))


Progress: |█| 100.0% Complete ETA:None s epoch=3: ELBO=543.6212,rec_loss=0.0 lat_loss=0.0
Epoch: 3, Test set ELBO: 544.892822265625, time elapse for current epoch: 24.142455339431763


<ipython-input-48-7578ee8870af>:6: UserWarning: image_at_epoch_0003.png is a low contrast image
  io.imsave('image_at_epoch_{:04d}.png'.format(epoch), prediction.astype(np.uint8))


Progress: |█| 100.0% Complete ETA:None s epoch=4: ELBO=543.7947,rec_loss=0.0 lat_loss=0.0
Epoch: 4, Test set ELBO: 544.6715698242188, time elapse for current epoch: 24.096307516098022
Progress: |█| 100.0% Complete ETA:None s epoch=5: ELBO=543.6124,rec_loss=0.0 lat_loss=0.0
Epoch: 5, Test set ELBO: 543.5389404296875, time elapse for current epoch: 24.09579849243164


...

Progress: |█| 100.0% Complete ETA:None s epoch=10: ELBO=543.9997,rec_loss=0.0 lat_loss=0.0
Epoch: 10, Test set ELBO: 544.7211303710938, time elapse for current epoch: 24.043173789978027


<ipython-input-48-7578ee8870af>:6: UserWarning: image_at_epoch_0010.png is a low contrast image
  io.imsave('image_at_epoch_{:04d}.png'.format(epoch), prediction.astype(np.uint8))


Progress: |█| 100.0% Complete ETA:None s epoch=11: ELBO=544.3544,rec_loss=0.0 lat_loss=0.0
Epoch: 11, Test set ELBO: 543.92041015625, time elapse for current epoch: 24.020835399627686
Progress: |█| 100.0% Complete ETA:None s epoch=12: ELBO=544.3468,rec_loss=0.0 lat_loss=0.0
Epoch: 12, Test set ELBO: 543.51953125, time elapse for current epoch: 23.97903347015381


<ipython-input-48-7578ee8870af>:6: UserWarning: image_at_epoch_0012.png is a low contrast image
  io.imsave('image_at_epoch_{:04d}.png'.format(epoch), prediction.astype(np.uint8))


KeyboardInterrupt: 

In [ ]:
model = tf.keras.Sequential()
model.add(layers.Dense(1024, input_shape=(400,)))
model.add(layers.Reshape((32, 32, 1)))
model.add(layers.UpSampling2D(size=(2, 2), interpolation='bilinear'))
model.add(RSU4F(256, 512))
model.add(layers.UpSampling2D(size=(2, 2), interpolation='bilinear'))
model.add(RSU4F(128, 256))
model.add(layers.UpSampling2D(size=(2, 2), interpolation='bilinear'))
model.add(RSU4(64, 128))
model.add(layers.UpSampling2D(size=(2, 2), interpolation='bilinear'))
model.add(RSU4(32, 64))
model.add(layers.Conv2DTranspose(16, kernel_size=(2, 2), strides=(2, 2)))
# model.add(RSU4(16, 1))
model.summary()